In [1]:
"""
Author: Elisha-Wigwe Chijioke O.
"""

%matplotlib inline
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt
import numpy as np
from pdb import set_trace
import pandas as pd

In [2]:
data_path = './pima.csv'
data = pd.read_csv(data_path)

In [3]:
data.head()

,preg,plas,pres,skin,test,mass,pedi,age,class
0,14,175,62,30,0,33.6,0.212,38,1
1,7,133,88,15,155,32.4,0.262,37,0
2,4,146,78,0,0,38.5,0.520,67,1
3,15,136,70,32,110,37.1,0.153,43,1
4,3,107,62,13,48,22.9,0.678,23,1


In [4]:
def error_fun(y, Y):
    return np.mean((y-Y)**2)

In [5]:
split_factor = 0.8
split = int(split_factor*data.shape[0])

train_data = data[:split]
test_data = data[split:]

target_fields = ['class']
features, targets = train_data.drop(target_fields, axis=1), train_data[target_fields]

In [6]:
test_features, test_targets = test_data.drop(target_fields, axis=1), test_data[target_fields]

In [7]:
val_split_factor = 0.7
val_split = int(val_split_factor*train_data.shape[0])

train_features, train_targets = features[:val_split], targets[:val_split]
val_features, val_targets = features[val_split:], targets[val_split:]

In [8]:
from pdb import set_trace
class NeuralNetwork:
    def __init__(self, input_nodes, hidden_nodes, output_nodes, learning_rate=1):
        self.input_nodes = input_nodes
        self.hidden_nodes = hidden_nodes
        self.output_nodes = output_nodes
        
        self.lr = learning_rate
        
        self.w_input_to_hidden = np.random.normal(0.0, self.input_nodes**-0.5, 
                                       (self.input_nodes, self.hidden_nodes))
        
        self.w_hidden_to_output = np.random.normal(0.0, self.hidden_nodes**-0.5, 
                                       (self.hidden_nodes, self.output_nodes))
        
        # activation function
        self.activation_fn = lambda x: 1.0/(1+np.exp(-x))
        self.activation_fn_deriv = lambda z: z*(1-z)
        
    
    # train
    def train(self, features, targets):
        n_records = features.shape[0]
        delta_weights_i_h = np.zeros(self.w_input_to_hidden.shape)
        delta_weights_h_o = np.zeros(self.w_hidden_to_output.shape)
        
        for x, y in zip(features, targets):
            f_out, h_out = self.forward_prop(x)
            delta_weights_i_h, delta_weights_h_o = self.backprop(f_out, h_out, x, y, 
                                                                        delta_weights_i_h, delta_weights_h_o)
        self.update_weights(delta_weights_i_h, delta_weights_h_o, n_records)
            
    
    def forward_prop(self, x):
        hidden_i = np.dot(x, self.w_input_to_hidden)
        hidden_o = self.activation_fn(hidden_i)
        
        final_i = np.dot(hidden_o, self.w_hidden_to_output)
        final_o = self.activation_fn(final_i)
        
        return final_i, final_o
    
    def backprop(self, final_o, hidden_o, x, y, delta_weights_i_h, delta_weights_h_o):
        
        error = y - final_o
        output_error_delta = error * self.activation_fn_deriv(final_o)
        
        hidden_error = np.dot(self.w_hidden_to_output, output_error_delta)
        hidden_error_delta = hidden_error * self.activation_fn_deriv(hidden_o)
        
        delta_weights_h_o = output_error_delta * hidden_o[:, None]
        delta_weights_i_h = hidden_error_delta * x[:, None]
        
        return delta_weights_i_h, delta_weights_h_o
    
    
    def update_weights(self, delta_weights_i_h, delta_weights_h_o, n_records):
        self.w_hidden_to_output += self.lr * delta_weights_h_o / n_records
        self.w_input_to_hidden += self.lr * delta_weights_i_h / n_records
        
    
    def run(self, features):
        hi = np.dot(features, self.w_input_to_hidden)
        ho = self.activation_fn(hi)
        
        fi = np.dot(ho, self.w_hidden_to_output)
        fo = self.activation_fn(fi)
        
        return fo
        

In [ ]:
import sys

epochs = 2000
learning_rate = 1e-2
hidden_nodes = 10
output_nodes = 1
input_nodes = train_features.shape[1]

network = NeuralNetwork(input_nodes, hidden_nodes, output_nodes, learning_rate)

In [ ]:
losses = { 'train':[], 'valid':[] }

for ii in range(epochs):
    batch = np.random.choice(train_features.index, size=20)
    x, y = train_features.ix[batch].values, train_targets.ix[batch]['class']
    
    # training
    network.train(x, y)
    
    train_loss = error_fun(network.run(train_features).T, train_targets['class'].values)
    val_loss = error_fun(network.run(val_features).T, val_targets['class'].values)
    
    sys.stdout.write("\rProgress: {:2.2f}".format(100 * ii/float(epochs)) \
              + "% ... Training loss: " + str(train_loss)[:5] \
              + " ... Validation loss: " + str(val_loss)[:5])
    sys.stdout.flush()
    
    losses['train'].append(train_loss)
    losses['valid'].append(val_loss)
    


Progress: 2.50% ... Training loss: 0.306 ... Validation loss: 0.242

/Users/andeladeveloper/anaconda3/envs/diab_proj/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """


Progress: 13.70% ... Training loss: 0.353 ... Validation loss: 0.276

In [ ]:
plt.plot(losses['train'], label='Training loss')
plt.plot(losses['valid'], label='Validation loss')
plt.legend()
_ = plt.ylim()

In [ ]:
# Run on testing features